In [12]:
# setup GCP to run queries
# TODO: change service account key to environment variable

from google.cloud import bigquery
client = bigquery.Client.from_service_account_json('/Users/KevinLubin/Desktop/ds/pearl/pearl-336700-0fa91569420d.json')

In [20]:
# Perform query
# TODO: import pandas biq query package and re-run query
# TODO: finalized query parameters (countries, time frame, etc.)

QUERY = """

    SELECT GlobalEventID, SQLDATE, EventRootCode, Quadclass, Actor1Type1Code, Actor2Type1Code, AvgTone,
    GoldSteinScale, NumMentions, Sourceurl

    FROM `gdelt-bq.full.events`

    WHERE (SQLDATE >= 20210101 AND SQLDATE <= 20210131)

    AND (Actor1CountryCode = 'DEU' OR Actor1CountryCode = 'JPN')

    AND (Actor2CountryCode = 'DEU' OR Actor2CountryCode = 'DEU')

    AND (Actor1CountryCode != Actor2CountryCode)

    GROUP BY SQLDATE, GlobalEventID, EventRootCode, Quadclass, Actor1Type1Code, Actor2Type1Code, AvgTone, 
    GoldSteinScale, NumMentions, Sourceurl
    
    ORDER BY SQLDATE, GlobalEventID, EventRootCode, Quadclass, Actor1Type1Code, Actor2Type1Code, AvgTone,
    GoldSteinScale, NumMentions, Sourceurl
    
"""

query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

for row in rows:
    print(row.EventRootCode)

04
04
02
02
06
06
11
11
04
04
02
02
01
01
01
11
11
11
16
16
04
04
04
01
01
01
01
13
04
04
04
01
01
04
08
07
11
05
05
05
05
05
05
04
04
04
04
04
04
04
04
04
04
04
08
08
08
16
16
17
11
11
04
04
04
04
04
04
04
04
04
04
01
01
02
04
04
04
05
05
05
04
04
02
02
02
02
04
02
02
02
02
02
02
02
02
02
02
04
09
09
02
04
04
11
11
05
05
05
